In [1]:
import sys # We need sys so that we can pass argv to QApplication

import design # This file holds our MainWindow and all design related things
              # it also keeps events etc that we defined in Qt Designer    
from PyQt5 import QtWidgets, QtCore, QtGui
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar)
import pandas as pd

In [3]:
sys.path.insert(0,'/home/janik/Praktikum_medPhys/FirstSearchAlgos/code/')
import util as ut

In [4]:
import imp
imp.reload(design)

<module 'design' from '/home/janik/Praktikum_medPhys/AnalyseOptimizer/design.py'>

In [5]:
class ExampleApp(QtWidgets.QMainWindow, design.Ui_AnalyzeCosts):
    def __init__(self,directory):
        # Simple reason why we use it here is that it allows us to
        # access variables, methods etc in the design.py file
        super(self.__class__, self).__init__()
        
        self.setupUi(self)  # This is defined in design.py file automatically
                            # It sets up layout and widgets that are defined
        
        self.dir = directory
        # add all plans to the plan list
        self.browse_folder(directory=self.dir)

   
        # connect clicked list item to bar plot
        self.plan_list.itemClicked.connect(self.FormRefList)

        self.comboBox_refplan.activated.connect(self.ChangeHbar)
        self.spinBox_uncert.valueChanged.connect(self.SpinBoxTrick)
        
        
        self.pushButton_gt.pressed.connect(self.PlotGT)
        self.pushButton_gc.pressed.connect(self.PlotGC)
        
        self.comboBox_organ1.activated.connect(self.PlotOrgan1HBar)
        self.comboBox_organ2.activated.connect(self.PlotOrgan2HBar)
        self.comboBox_organ3.activated.connect(self.PlotOrgan3HBar)
        
        
        self.checkBox_organUpdate.stateChanged.connect(self.UpdateOrgans)
        #self.pushButton_plot.setEnabled(False)
        #self.pushButton_plot.pressed.connect(self.ChangeHbar)
        
        self.pushButton_save.setEnabled(False)
        
        # Create an empty figure
        self.fig_Hbar = Figure()
        self.canvas_Hbar = FigureCanvas(self.fig_Hbar)
        
        self.fig_Organ1 = Figure()
        self.canvas_Organ1 = FigureCanvas(self.fig_Organ1)

        self.fig_Organ2 = Figure()
        self.canvas_Organ2 = FigureCanvas(self.fig_Organ2)
        
        self.fig_Organ3 = Figure()
        self.canvas_Organ3 = FigureCanvas(self.fig_Organ3)
        
        self.figGT = Figure()
        self.canvasGT = FigureCanvas(self.figGT)
        self.UpdateGT = False
        
        self.figGC = Figure()
        self.canvasGC = FigureCanvas(self.figGC)
        self.UpdateGC = False
        
        #self.ShowHbar(fig_Hbar)
        self.HbarSummary.addWidget(self.canvas_Hbar)
        self.organ1.addWidget(self.canvas_Organ1)
        self.organ2.addWidget(self.canvas_Organ2)
        self.organ3.addWidget(self.canvas_Organ3)
        self.gtTracks.addWidget(self.canvasGT)
        self.gcTracks.addWidget(self.canvasGC)
        
        self.ClearPlots()
        self.colors = ['darkorange','blue','green','yellow','navy','lime',
                     'gray','cyan','magenta','firebrick','khaki','peru']
    
    def SpinBoxTrick (self) :
        self.spinBox_uncert.setEnabled(False)
        self.ChangeHbar()
    
    def UpdateOrgans (self) :
        if self.checkBox_organUpdate.isTristate() :
            self.checkBox_organUpdate.setTristate(False)
        else :
            self.checkBox_organUpdate.setTristate(True)
    
    def PlotAxis (self,axis,key,color) :
        ax = axis
        
        #print(ax.prop_cycle())
        values = [x[key] for x in self.costs]
        
        ax.barh(bottom=np.arange(len(values)),width=values,tick_label=self.plans,color=color,
               edgecolor='black')
        
        reference_name = self.comboBox_refplan.currentText()     
        reference_id = self.df[self.df['name']==reference_name].index[0]
        
        ax.vlines(values[reference_id],-0.5,len(values)-0.5,'r',linestyle='--',label='reference')

        ax.legend(loc='upper right', borderaxespad=0)
        ax.set_xlabel('cost [a.u.]')
        ax.set_ylabel('plan name')
    
    def PlotOrgan1HBar (self) :
        ax = self.fig_Organ1.add_subplot(111)
        ax.clear()        
        
        self.fig_Organ1.subplots_adjust(left=0.3)       
        key = self.comboBox_organ1.currentText()
        color_index = self.comboBox_organ1.currentIndex()
        self.PlotAxis(axis=ax, key=key,color=self.colors[color_index])      
        self.canvas_Organ1.draw()
    
    def PlotOrgan2HBar (self) :
        ax = self.fig_Organ2.add_subplot(111)
        ax.clear()        
        
        self.fig_Organ2.subplots_adjust(left=0.3)       
        key = self.comboBox_organ2.currentText()
        color_index = self.comboBox_organ2.currentIndex()
        self.PlotAxis(axis=ax, key=key,color=self.colors[color_index])      
        self.canvas_Organ2.draw()

    def PlotOrgan3HBar (self) :
        ax = self.fig_Organ3.add_subplot(111)
        ax.clear()        
        
        self.fig_Organ3.subplots_adjust(left=0.3)       
        key = self.comboBox_organ3.currentText()
        color_index = self.comboBox_organ3.currentIndex()
        self.PlotAxis(axis=ax, key=key,color=self.colors[color_index])      
        self.canvas_Organ3.draw()
    """    
    def ActivatePlot (self) :
        self.pushButton_plot.setEnabled(True)
        #clear organ plots
        self.ClearPlots()
    """    
    def ClearPlots (self) :    
        ax = self.fig_Organ1.add_subplot(111)
        ax.clear()
        self.canvas_Organ1.draw()
        ax = self.fig_Organ2.add_subplot(111)
        ax.clear()
        self.canvas_Organ2.draw()
        ax = self.fig_Organ3.add_subplot(111)
        ax.clear()
        self.canvas_Organ3.draw()
        
        ax = self.figGT.add_subplot(111)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)        
        ax.clear()
        self.canvasGT.draw()

        ax = self.figGC.add_subplot(111)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)        
        ax.clear()
        self.canvasGC.draw()

        #ax = self.fig_Hbar.add_subplot(111)
        #ax.clear()
        #self.canvas_Hbar.draw()


        
    def browse_folder(self,directory):
        self.plan_list.clear()
        self.plan_list.setSortingEnabled(True)
        if directory:
            for file_name in os.listdir(directory): 
                self.plan_list.addItem(file_name)
    
    def FormRefList (self) :
        self.comboBox_refplan.clear()
        for item in self.plan_list.selectedItems() :
            self.comboBox_refplan.addItem(item.text())
        
        self.ChangeHbar()
        print('ref list')
        #print('current text', self.comboBox_refplan.currentText())
        #self.ClearPlots()
    def ChangeHbar(self):
        self.progressBar_plotting.setValue(0)
        #print('update', self.checkBox_organUpdate.isTristate())
        if self.checkBox_organUpdate.isTristate() == False :
            self.ClearPlots()
        
        self.plans = [item.text() for item in self.plan_list.selectedItems()]
        self.costs = GetCosts(pfx=self.dir,plans=self.plans)
        self.df = pd.DataFrame(data=self.costs)
        self.df['name'] = self.plans
        
        #df = dataframe
        df2 = self.df.iloc[:,:-2]
        #analysis_name = name
        perfect_obj = []
        for k in df2.keys() :
            if (np.all(df2[k]==0)) :
                #print('objective %s can be fully satisfied'%(k))
                perfect_obj.append(k)
        df_plot = df2.drop(perfect_obj,axis=1)
        self.progressBar_plotting.setValue(20)

        
        ax = self.fig_Hbar.add_subplot(111)
        ax.clear()
        
        df_plot.plot(kind='barh',stacked=True,use_index=False,yticks=['a'],ax=ax,color=self.colors)
        self.progressBar_plotting.setValue(30)
        
        ax.set_yticklabels(self.df.name)
        ax.set_xlabel('cost [a.u.]')
        ax.set_ylabel('plan name')
        
        #print(self.comboBox_refplan.currentText())
        ref_plan = self.comboBox_refplan.currentText()
        reference = (self.df[self.df['name']==ref_plan]['TOTAL'])
        reference = reference.values[0]

        percentage = int(self.spinBox_uncert.value())
        #print(percentage)
        
        ax.vlines(reference,-0.5,len(self.df['TOTAL'])-0.5,'r',linestyle='--',label=str(percentage)+' % uncert.')
        
        ax.axvspan(reference*(1.-percentage/100.),reference*(1.+percentage/100.),alpha=0.5,color='red')
        self.progressBar_plotting.setValue(40)
        
        ax.legend(bbox_to_anchor=(1.11,1.01), borderaxespad=0,fontsize=14)
        
        
        self.canvas_Hbar.draw()
        
        self.FillOrganSelection(names=df_plot.keys())
        self.progressBar_plotting.setValue(50)

        #update organ plots
        if self.checkBox_organUpdate.isTristate() == True :
            self.PlotOrgan1HBar()
            self.PlotOrgan2HBar()
            self.PlotOrgan3HBar()
        
        self.progressBar_plotting.setValue(100)
        self.spinBox_uncert.setEnabled(True)
        
        
    def FillOrganSelection (self, names) :
        if (self.checkBox_organUpdate.isTristate() == False) :
            self.comboBox_organ1.clear()
            self.comboBox_organ2.clear()
            self.comboBox_organ3.clear()
            for item in names :
                self.comboBox_organ1.addItem(item)
                self.comboBox_organ2.addItem(item)
                self.comboBox_organ3.addItem(item)
        else :
            o1_selected = self.comboBox_organ1.currentText()
            o2_selected = self.comboBox_organ2.currentText()
            o3_selected = self.comboBox_organ3.currentText()
            
            
            self.comboBox_organ1.clear()
            self.comboBox_organ2.clear()
            self.comboBox_organ3.clear()
            for item in names :
                self.comboBox_organ1.addItem(item)
                self.comboBox_organ2.addItem(item)
                self.comboBox_organ3.addItem(item)
            
            self.comboBox_organ1.setCurrentText(o1_selected)
            self.comboBox_organ2.setCurrentText(o2_selected)
            self.comboBox_organ3.setCurrentText(o3_selected)
            
    def PlotGT (self) :   
        #self.pushButton_gt.setEnabled(False)
        self.pushButton_gt.setText('update gt')
        if self.UpdateGT :
            for j in range(self.n) :
                ax = self.figGT.add_subplot(str(1)+str(self.n)+str(j+1))
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                ax.clear()
                self.canvasGT.draw()

        
        N_plans = len(self.plans)
        self.n = np.maximum(4,N_plans)
        self.UpdateGT = True
        
        for i,p in enumerate(self.plans) :
            ax = self.figGT.add_subplot(str(1)+str(self.n)+str(i+1))
            ax.clear()
        
            print('plan',p)
            File = open(self.dir+p+'/plan.txt')
            track_names = (File.readlines()[3].split('=')[1].split(' '))
            print(track_names)
            if '\n' in track_names :
                track_names = list(track_names[:-1])
                if '' in track_names :
                    track_names = list(track_names[:-1])
            File.close()

            print('track name ',track_names)
            
            
            for t in track_names :
                print(t)
                if '\n' in t :
                    t = t[:-1]
                nFile = open(self.dir+p+'/gtc/'+t+'-gtc.txt')

                ga = []
                ta = []
                ca = []
                for f in nFile.readlines()[1:] :
                    value = (f.split('\t'))
                    ga.append(float(value[0]))
                    ta.append(float(value[1]))
                    ca.append(float(value[2][:-1]))
                    #print((value[2][:-1]))
                nFile.close()
                
                ax.plot(ga,ta,'k',lw=8)
                ax.plot(ga,ta,'r',lw=4)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                
            
            dim, gga, tta, col_map = ut.load3DColorwash(path=self.dir+p+'/gt-maps/'+t+'-map.txt')
            X,Y = np.meshgrid(gga,tta)
            ax.pcolor(X,Y,col_map,cmap='jet')
            ax.set_title('Plan: '+p)

        self.canvasGT.draw()
        print('done')
            
    def PlotGC (self) :   
        #self.pushButton_gt.setEnabled(False)
        self.pushButton_gc.setText('update gc')
        if self.UpdateGC :
            for j in range(self.nGC) :
                ax = self.figGC.add_subplot(str(1)+str(self.nGC)+str(j+1))
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                ax.clear()
                self.canvasGC.draw()

        
        N_plans = len(self.plans)
        self.nGC = np.maximum(4,N_plans)
        self.UpdateGC = True
        
        for i,p in enumerate(self.plans) :
            ax = self.figGC.add_subplot(str(1)+str(self.nGC)+str(i+1))
            ax.clear()
        
            print('plan',p)
            File = open(self.dir+p+'/plan.txt')
            track_names = (File.readlines()[3].split('=')[1].split(' '))
            print(track_names)
            if '\n' in track_names :
                track_names = list(track_names[:-1])
                if '' in track_names :
                    track_names = list(track_names[:-1])
            File.close()

            print('track name ',track_names)
            
            
            for t in track_names :
                print(t)
                if '\n' in t :
                    t = t[:-1]
                nFile = open(self.dir+p+'/gtc/'+t+'-gtc.txt')

                ga = []
                ta = []
                ca = []
                for f in nFile.readlines()[1:] :
                    value = (f.split('\t'))
                    ga.append(float(value[0]))
                    ta.append(float(value[1]))
                    ca.append(float(value[2][:-1]))
                    #print((value[2][:-1]))
                nFile.close()
                
                ax.plot(ga,ca,'k',lw=8)
                ax.plot(ga,ca,'r',lw=4)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                
            
                dim, gga, cca, col_map = ut.load3DColorwash(path=self.dir+p+'/gc-maps/'+t+'-gap.txt')
                X,Y = np.meshgrid(gga,cca)
                ax.pcolor(X,Y,col_map,cmap='jet')
                ax.set_title('Plan: '+p)

        self.canvasGC.draw()
        print('done')
            
                   

In [6]:
def GetCosts (pfx, plans) :
    costs = []
    for p in plans :
        opt_cost = pfx+'/'+p+'/opt/objectives_cost.txt'
        File = open(opt_cost,'r')
        cost_dict = {}
        for f in File.readlines()[:-1] :
            word = f.split(' = ')
            cost_dict[word[0]] = float(word[1])
        costs.append(cost_dict)
        File.close()
    return costs

In [7]:
def main (pathToPlans) :
    app = QtWidgets.QApplication(sys.argv)  
    form = ExampleApp(directory=pathToPlans)              
    form.show()                        
    app.exec_()  

In [8]:
pathToPlans = '/home/janik/Praktikum_medPhys/dto/patients/Camus/plans/'
main(pathToPlans=pathToPlans)

ref list
plan AstarVoxel5
['t_m180_p0', 't_p45_p180\n']
track name  ['t_m180_p0', 't_p45_p180\n']
t_m180_p0
t_p45_p180

done
plan AstarVoxel5
['t_m180_p0', 't_p45_p180\n']
track name  ['t_m180_p0', 't_p45_p180\n']
t_m180_p0
t_p45_p180

done
ref list
ref list
ref list
ref list
ref list
ref list
ref list
ref list
plan GC2
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC2_2
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC3
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC1
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan p1
['t1', '\n']
track name  ['t1']
t1


FileNotFoundError: [Errno 2] No such file or directory: '/home/janik/Praktikum_medPhys/dto/patients/Camus/plans/p1/gtc/t1-gtc.txt'

plan GC2
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC2_2
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC3
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC1
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan p1
['t1', '\n']
track name  ['t1']
t1
done
plan GC2
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC2_2
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC3
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan GC1
['t1', 't2', '\n']
track name  ['t1', 't2']
t1
t2
plan p1
['t1', '\n']
track name  ['t1']
t1
done
